In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import os
from Bio.Blast import NCBIXML
import matplotlib.pyplot as plt
from BCBio import GFF
from pprint import pprint
import dill
from IPython.display import FileLink
import xml.etree.ElementTree as ET
import stopwatch
from __future__ import division
%matplotlib inline
from Bio import SeqFeature
from Bio.SeqFeature import FeatureLocation, ExactPosition
from os import path as op
from os import listdir as ls
from Bio.Blast.Applications import NcbiblastxCommandline

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: license expired

    the package numpy 1.10.2 was published on 2015-12-14,
    however the mkl license expiration date is 2015-05-28.
    You may be able to run earlier versions of numpy using your
    current license.  A new license can be purchased at: http://continuum.io
    To revert to an earlier set of conda packages, use:
    $ conda list --revisions
    ...
    $ conda install --revision <REVISION NUMBER>

    


In [32]:
#get my SNPs
filE = '/home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE.txt'
df = pd.read_csv(filE,header=0,index_col=0,sep='\t')
df = pd.DataFrame(df.T)
snps = list(np.unique(df.columns))
contigs = set(["_".join(snp.split("_")[:-1]) for snp in snps])

In [24]:
assembly = '/home/lindb/wbp/concatenated/velvet/auto_data_45/contigs.fa'

In [35]:
count = 0
out = open(op.join(op.dirname(assembly), "contigs_with_snps.fa"), "w")
reads = []
for read in SeqIO.parse(assembly, "fasta"):
    if read.id in contigs:
        reads.append(read)
        count += 1
SeqIO.write(reads, out, "fasta")
out.close()
print out.name

/home/lindb/wbp/concatenated/velvet/auto_data_45/contigs_with_snps.fa


In [36]:
!grep -c ">" {out.name}

/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `BASH_FUNC_module'
116231


In [37]:
#make BLAST db
!~/g/src/ncbi-blast-2.3.0+/bin/makeblastdb -in \
/home/lindb/eckertlab/SugarPine_genome/pila.v1.0.scafSeq -dbtype nucl

In [10]:
cmd = '/home/lindb/g/src/ncbi-blast-2.3.0+/bin/blastn -out /home/lindb/wbp/blast/wbp_blast_116231snps_out2.txt -outfmt 6 \
-query /home/lindb/wbp/blast/contigs_with_snps.fa -evalue 1e-05 -max_target_seqs 5 \
-num_threads 20 -db /home/lindb/eckertlab/SugarPine_genome/pila.v1.0.scafSeq' 

In [11]:
outdir = '/home/lindb/wbp/blast/'
with open(os.path.join(outdir, "blast.sh"), "w") as o:
    o.write("%s\n" % cmd)

    ~/bin/parallel --no-notice --bar -a blast.sh

In [17]:
df=pd.read_csv('/home/lindb/wbp/blast/wbp_blast_116231snps_out2.txt',header=None,sep='\t')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,NODE_162_length_98_cov_5.010204,scaffold522278,100.000,139,0,0,4,142,1689,1827,8.950000e-66,257.0
1,NODE_162_length_98_cov_5.010204,scaffold503446,100.000,139,0,0,4,142,1703,1841,8.950000e-66,257.0
2,NODE_162_length_98_cov_5.010204,scaffold338276,100.000,139,0,0,4,142,957,1095,8.950000e-66,257.0
3,NODE_162_length_98_cov_5.010204,scaffold52130,100.000,139,0,0,4,142,3445,3583,8.950000e-66,257.0
4,NODE_162_length_98_cov_5.010204,scaffold24443,99.291,141,1,0,2,142,2141,2001,3.220000e-65,255.0


In [18]:
len(np.unique(df[0]))

107354

In [19]:
len(df.index)

582469

In [20]:
len(np.unique(df[1]))

167363

In [3]:
timer = stopwatch.Timer()
query_percs = []
id_percs = []
good_hits  = 0
good_queries = 0
query_min = 50
id_min = 75
total_recs = 0
query_lens = []
good_hit_info = {'len':[], 'len_perc':[], 'id_perc':[]}
for i, record in enumerate(NCBIXML.parse(open(blast_file))):
    if len(record.alignments) > 0:
        snp_data.loc[record.query,"hits"] = ""
        hit_defs = set()
        for aln in record.alignments:
            for hsp in aln.hsps:
                query_perc = ((hsp.query_end - hsp.query_start)+1)*100.0/record.query_length
                query_percs.append(query_perc)
                query_lens.append(record.query_length)
                id_perc = hsp.identities*100.0/hsp.align_length
                id_percs.append(id_perc)
                if (query_perc >= query_min) and (id_perc >= id_min):
                    hit_defs.add("%s:%d:%d" % (aln.hit_def, hsp.sbjct_start, hsp.sbjct_end))
                    good_hits += 1
                    good_hit_info['len'].append(record.query_length)
                    good_hit_info['len_perc'].append(query_perc)
                    good_hit_info['id_perc'].append(id_perc)
        if len(hit_defs) > 0:
            good_queries += 1
            snp_data.loc[record.query,"hits"] = "|".join(hit_defs)
    total_recs += 1
#     if i == 100:
#         break

#make sure blank columns are nan
snp_data.hits = snp_data.hits.replace("", np.nan)

timer.stop()
print "found %d good hits (%d queries) out of %d records (%.2f%%) in %s " % (good_hits, 
                                                                             good_queries,
                                                                             total_recs, 
                                                                             good_queries*100.0/total_recs, 
                                                                             timer)

ValueError: Your XML file did not start with <?xml... but instead 'super1\tNODE_961681_l'